In [52]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Path to the directory containing the droplet images
image_dir = "./Data"

# List to store the preprocessed images and corresponding labels
preprocessed_images = []
labels = []

# Iterate through the image files in the directory
for filename in os.listdir(image_dir):
    if filename.endswith(".png"):
        # Extract the tension value from the filename
        tension = int(os.path.splitext(filename)[0])

        # Read the image using OpenCV
        image_path = os.path.join(image_dir, filename)
        image = cv2.imread(image_path)

        # Preprocess the image (e.g., resize, convert to grayscale, normalize)
        image = cv2.resize(image, (256, 256))  # Resize the image to a consistent size
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        image = image.astype(np.float32) / 255.0  # Normalize pixel values to the range [0, 1]

        # Append the preprocessed image and its label to the lists
        preprocessed_images.append(image)
        labels.append(tension)

# Convert the image and label lists to NumPy arrays
preprocessed_images = np.array(preprocessed_images)
labels = np.array(labels)

# Print the shapes of the resulting datasets
print("Train images shape:", preprocessed_images.shape)
print("Train labels shape:", labels.shape)


Train images shape: (7, 256, 256)
Train labels shape: (7,)


In [53]:
# Define the CNN architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # Assuming you want to predict a single tension value
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Print the model summary
# model.summary()


In [54]:
# Convert labels to float32
labels = labels.astype(np.float32)

# Reshape the images to match the input shape of the model
preprocessed_images = preprocessed_images.reshape(-1, 256, 256, 1)

# Train the model
model.fit(preprocessed_images, labels, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 5861.3311 - val_loss: 3160.9204
Epoch 2/10
1/1 [==============================] - 0s 330ms/step - loss: 2385.8835 - val_loss: 271.0258
Epoch 3/10
1/1 [==============================] - 0s 369ms/step - loss: 191.2429 - val_loss: 1050.5328
Epoch 4/10
1/1 [==============================] - 0s 348ms/step - loss: 1572.2001 - val_loss: 1077.3131
Epoch 5/10
1/1 [==============================] - 0s 399ms/step - loss: 1596.1873 - val_loss: 290.7196
Epoch 6/10
1/1 [==============================] - 0s 447ms/step - loss: 633.3465 - val_loss: 25.6957
Epoch 7/10
1/1 [==============================] - 0s 373ms/step - loss: 140.6586 - val_loss: 278.2020
Epoch 8/10
1/1 [==============================] - 0s 349ms/step - loss: 196.2474 - val_loss: 635.7351
Epoch 9/10
1/1 [==============================] - 0s 351ms/step - loss: 421.8500 - val_loss: 855.8518
Epoch 10/10
1/1 [==============================] - 0s 355ms/step - loss: 578.54

In [55]:
# Read and preprocess the input image
input_image_path = "./Data/90.png"
input_image = cv2.imread(input_image_path)
input_image = cv2.resize(input_image, (256, 256))
input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
input_image = input_image.astype(np.float32) / 255.0
input_image = np.expand_dims(input_image, axis=0)  # Add an extra dimension for batch size

# Perform prediction
prediction = model.predict(input_image)

# Retrieve the predicted tension value
predicted_tension = prediction[0]
print("Predicted tension:", predicted_tension)


1/1 [==============================] - 0s 86ms/step
Predicted tension: [56.012547]
